In [10]:
import pandas as pd
import csv
from collections import OrderedDict

input_file = csv.DictReader(open("fake_review_train.csv",encoding='cp850'))
data = []
k = 0
for i in input_file:
    data.append(dict(i))

# data = pd.read_csv("datasets\data2.csv")
# print(type(d),d)

# data1 = pd.read_csv('datasets\data2.csv', header=None, index_col=0, squeeze=True).to_dict()
# print(type(data),data,len(data))

print(data[0])

{'index': '0', 'ID': 'ID-0975', 'Review': "I mistakenly thought that since my multiple stays at other Marriott locations were excellent and clean that this location would be as well. Man was I wrong. This place is terrible. The room was very dirty and there were dead bugs everywhere. The room smelled horrible. I couldn't get the room windows open enough to help remove the smell. It took a while to find this place with all the construction in the area and the awkward road design in the area. I will never stay here again!\n", 'Label': 'True'}


In [2]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
#         print("1")
#         print(list_of_tweets)
#         for i in list_of_tweets:
#             print(i["text"])
#             break
            
        for tweet in list_of_tweets:
#             print("22")
            processedTweets.append((self._processTweet(tweet["text"]),tweet["Classfication"]))
#             print(processedTweets)
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(data)

In [4]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [5]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [9]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

In [7]:
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [8]:
import pickle
f = open('nb.pickle', 'wb')
pickle.dump(NBayesClassifier, f)
f.close()

In [6]:
import pickle
f = open('nb.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [7]:
import random
test = []
result = []
for i in range(1000):
    t = random.choice(preprocessedTrainingSet)
    test.append(t[0])
    result.append(t[1])
    

# print(test,result)

In [10]:
k = 0
p = 0
for i in test:

    NB = NBayesClassifier.classify(extract_features(i))
    if NB == result[k]:
        p = p + 1
    k = k + 1

print(p*100/k)


91.6


In [11]:
print(p*100/k)

91.6
